In [ ]:
!pip install python-dotenv

In [ ]:
PROJECT_ID = "watchful-lotus-422412-h7"
REGION = "us-central1"

In [ ]:
from google.cloud import aiplatform
import google
from vertexai import preview
import os
from dotenv import load_dotenv
from google.oauth2 import service_account
load_dotenv()
from vertexai.vision_models import (
    MultiModalEmbeddingModel,
    MultiModalEmbeddingResponse,
    Video,
    VideoSegmentConfig,
)

from vertexai.language_models import TextEmbeddingModel
#creds = os.getenv("GOOGLE_APPLICATION_CREDENTIALS")
import vertexai
from vertexai.vision_models import (
    Image,
    MultiModalEmbeddingModel,
    MultiModalEmbeddingResponse,
)
from typing import Optional
## SETUP
credentials = service_account.Credentials.from_service_account_file('/content/watchful-lotus-422412-h7-a2ea8eb16a41.json')
#print(creds)
aiplatform.init(

    project='watchful-lotus-422412-h7',

    location='us-central1',

    experiment='my-experiment',
    credentials=credentials,

    experiment_description='my experiment description'
)

vertexai.init(project="watchful-lotus-422412-h7", location='us-central1')

In [ ]:
class Embedding_Generator:

  def text_embedding(self,text) -> list:
        """Text embedding with a Large Language Model."""
        model = TextEmbeddingModel.from_pretrained("textembedding-gecko@001")
        embeddings = model.get_embeddings([text])
        for embedding in embeddings:
            vector = embedding.values
            print(f"Length of Embedding Vector: {len(vector)}")
        return vector

  def get_image_embeddings(self,
        image_path: str,
        contextual_text: Optional[str] = None,
        dimension: int = 768,
    ) -> MultiModalEmbeddingResponse:
        """Example of how to generate multimodal embeddings from image and text.

        Args:
            project_id: Google Cloud Project ID, used to initialize vertexai
            location: Google Cloud Region, used to initialize vertexai
            image_path: Path to image (local or Google Cloud Storage) to generate embeddings for.
            contextual_text: Text to generate embeddings for.
            dimension: Dimension for the returned embeddings.
                https://cloud.google.com/vertex-ai/docs/generative-ai/embeddings/get-multimodal-embeddings#low-dimension
        """
        model = MultiModalEmbeddingModel.from_pretrained("multimodalembedding")
        image = Image.load_from_file(image_path)

        embeddings = model.get_embeddings(
            image=image,
            contextual_text=contextual_text,
            dimension=dimension,
        )
        return embeddings

  def get_video_embeddings(self,
        video_path: str,
        contextual_text: Optional[str] = None,
        dimension: Optional[int] = 1408,
        video_segment_config: Optional[VideoSegmentConfig] = None,
    ) -> MultiModalEmbeddingResponse:
        """Example of how to generate multimodal embeddings from video and text.

        Args:
            project_id: Google Cloud Project ID, used to initialize vertexai
            location: Google Cloud Region, used to initialize vertexai
            video_path: Path to video (local or Google Cloud Storage) to generate embeddings for.
            contextual_text: Text to generate embeddings for.
            dimension: Dimension for the returned embeddings.
                https://cloud.google.com/vertex-ai/docs/generative-ai/embeddings/get-multimodal-embeddings#low-dimension
            video_segment_config: Define specific segments to generate embeddings for.
                https://cloud.google.com/vertex-ai/docs/generative-ai/embeddings/get-multimodal-embeddings#video-best-practices
        """

        #vertexai.init(project=project_id, location=location)

        model = MultiModalEmbeddingModel.from_pretrained("multimodalembedding")
        video = Video.load_from_file(video_path)

        embeddings = model.get_embeddings(
            video=video,
            video_segment_config=video_segment_config,
            contextual_text=contextual_text,
            dimension=dimension,
        )

        # Video Embeddings are segmented based on the video_segment_config.
        print("Video Embeddings:")
        for video_embedding in embeddings.video_embeddings:
            print(
                f"Video Segment: {video_embedding.start_offset_sec} - {video_embedding.end_offset_sec}"
            )
            print(f"Embedding: {video_embedding.embedding}")

In [ ]:
!pip install faiss-cpu

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 27.0/27.0 MB 35.0 MB/s eta 0:00:00


In [ ]:
from google.colab import auth
auth.authenticate_user()

import gspread
from google.auth import default
creds, _ = default()

gc = gspread.authorize(creds)

import pandas as pd
import faiss
import numpy as np
import openpyxl
import yaml
import json
# read data and put it in a dataframe
gsheets = gc.open_by_url('https://docs.google.com/spreadsheets/d/1N8RFD-0baJBfmoFKc6tSCYEZdkC22IyptHHanZFhzc0/edit?pli=1#gid=0')
sheets = gsheets.worksheet('data').get_all_values()
df = pd.DataFrame(sheets[1:], columns=sheets[0])

In [ ]:
df

,Question,Answer - Text,Image,Image Caption,Video,Video-Transcript,Source
0,What does ROM stand for?,Read-only Memory (ROM) is a non-volatile memor...,,,,,Teachers Notes
1,Can you list the different types of ROM?,- PROM (Programmable ROM): PROM is a type of R...,,,,,Teachers Notes
2,What is PROM and how is it programmed?,"PROM, or Programmable ROM, is a type of ROM th...",,,,,Teachers Notes
3,What is EPROM (erasable programmable ROM),EPROM was invented to allow making changes in ...,,,,,Teachers Notes
4,What is UV-EPROM ?,"A widely used EPROM is called UV-EPROM, where ...",,,,,Teachers Notes
5,How does EPROM differ from PROM?,"EPROM, or Erasable Programmable ROM, can be er...",,,,,Teachers Notes
6,How to program a UV-EPROM,"To program a UV-EPROM chip, the following step...",,,,,Teachers Notes
7,What is EEPROM (electrically erasable programm...,"EEPROM, or Electrically Erasable Programmable ...",,,,,Teachers Notes
8,What are the advantages of EEPROM over EPROM?,EEPROM can be erased and programmed electrical...,,,,,Teachers Notes
9,What is Flash Memory EPROM,"Since the early 1990s, Flash EPROM has become ...",,,,,Teachers Notes


In [ ]:
df[-6:-2].columns


Index(['Question', 'Answer -  Text', 'Image', 'Image Caption ', 'Video ',
       'Video-Transcript ', 'Source'],
      dtype='object')

In [ ]:
data = {
    "question":"",
    "answer":"",
    "image_url":"",
    "image_caption":"",
    "video_url":"",
    "video_transcript":"",
    "source":""
}
data_map = []

for index, row in df[-6:-2].iterrows():
  print(index)
  data['question'] = row['Question']
  data['answer'] = row['Answer -  Text']
  data['image_url'] = row['Image']
  data['image_caption'] =row['Image Caption ']
  data['video_url']=row['Video ']
  data['video_transcipt']=row['Video-Transcript ']
  data['source'] = row['Source']
  data_map.append(data)
  data = {}

37
38
39
40


In [ ]:
data_map[0]['image_url'] = "/content/elgamal_cryptosystem.png"
data_map[-1]['image_url'] = "/content/aes_sub_bytes.jpg"

In [ ]:

len(data_map)

4

In [ ]:
import time
vectors = faiss.IndexFlatIP(1536)
text_map = []
image_map = []
idx = 0
eg = Embedding_Generator()
for row in data_map:
    retry = True
    while retry:
        try:
            text_embd = eg.text_embedding(row['question'])
            retry = False
        except Exception as e:
            print(e)
            time.sleep(10)
    text_map.append(text_embd)
    print(f"processed {idx} rows")
    idx += 1



Length of Embedding Vector: 768
processed 0 rows
Length of Embedding Vector: 768
processed 1 rows
Length of Embedding Vector: 768
processed 2 rows
Length of Embedding Vector: 768
processed 3 rows


In [ ]:
vectors = faiss.IndexFlatIP(768)
np_list = np.asarray(text_map)
vectors.add(np_list)
faiss.write_index(vectors, f"vectors.db")
yaml.dump(data_map, open(f"datamap.yaml", "w"))

In [ ]:
query_question = "Sub Bytes AES"
text_query_embed = eg.text_embedding(query_question)
dist, indx = vectors.search(
        np.asarray([text_query_embed], dtype=np.float32), k=2
    )
indexes = indx[0]
distances = dist[0]
iamge_links = []
context = "\n\n".join(
        [
            "\n Source : " +data_map[indexes[i]]['source'] + data_map[indexes[i]]["answer"]
            for i in range(len(indexes)) if distances[i] >= 0.5
        ]
    )

Length of Embedding Vector: 768


In [ ]:
context

'\n Source : foruzanSubBytes (S)\r\n{\r\n for (r = 0 to 3)\r\n for (c = 0 to 3)\r\n Sr,c = subbyte (Sr,c)\r\n}\r\nsubbyte (byte)\r\n{\r\n a ← byte−1 // Multiplicative inverse in GF(28\r\n) with inverse of 00 to be 00 \r\n ByteToMatrix (a, b)\r\n for (i = 0 to 7) \r\n {\r\n ci ← bi ⊕ b(i+4)mod 8 ⊕ b(i+5)mod 8 ⊕ b(i+6)mod 8 ⊕ b(i+7)mod 8\r\n di ← ci ⊕ ByteToMatrix (0x63)\r\n } \r\n MatrixToByte (d, d)\r\n byte ← d \r\n} \r\n\n\n Source : william stallings 1. Initialize the S-box with the byte values in ascending sequence row by row.\r\nThe  first row contains {00}, {01}, {02}, c , {0F}; the second row contains\r\n{10}, {11}, etc.; and so on. Thus, the value of the byte at row y, column x is {yx}.\r\n2. Map each byte in the S-box to its multiplicative inverse in the finite field\r\nGF(28\r\n); the value {00} is mapped to itself.\r\n3. Consider that each byte in the S-box consists of 8 bits labeled\r\n(b7, b6, b5, b4, b3, b2, b1, b0). Apply the following transformation to each bit of\r\nea

In [ ]:
from vertexai.generative_models import (
    GenerationConfig,
    GenerativeModel,
    Image,
    Part,
)
multimodal_model = GenerativeModel("gemini-1.0-pro-vision")
question = query_question  # Replace with the actual student's question
context =context
image = "/content/aes_sub_bytes.jpg"
prompt = f"""A student is studying and has a question about the process of {question}.
They are looking at an image that depicts "{question} in the {context}".

Answer the student's question in detail, combining information from different sources and using all of it.
Explain each part in the provided image with the help of descriptions.
Make sure to include all equations and explain all examples.
"""
contents = [image, prompt]

responses = multimodal_model.generate_content(contents, stream=True)

print("-------Prompt--------")
print("\n-------Response--------")
for response in responses:
    print(response.text, end="")

-------Prompt--------

-------Response--------
**Sub Bytes Transformation in AES**

The Sub Bytes transformation is a critical step in the Advanced Encryption Standard (AES) algorithm that enhances data security. It involves replacing each byte in the input state with a corresponding byte from a predefined substitution table known as the S-box. This transformation aims to introduce non-linearity into the cipher, making it harder to analyze and cryptanalyze.

**Details of the Provided Image:**

The image you provided contains two different descriptions of the Sub Bytes transformation:

**Source 1 (Forouzan):**

* The Sub Bytes transformation is performed iteratively for each row and column of the input state.
* The `subbyte()` function takes a byte as input and performs the following steps:
    * Multiplicative inverse is computed for the input byte in the Galois Field GF(28).
    * The input byte is converted into a 2D matrix.
    * For each bit position (i) in the matrix:
        * `c

In [ ]:

import vertexai
from vertexai.preview.generative_models import GenerativeModel, Image, Part
video_url = "gs://desvideo/DES Algorithm _ Working of DES Algorithm _ DES Encryption Process _ Data Encryption Standard.mp4"

prompt = """
Answer the following questions stritly using the video only.
Answer from the knowledge given in the video only.
Do not add your own information .:
can you describe the steps of des algorithm.
"""

video = Part.from_uri(video_url, mime_type="video/mp4")
contents = [prompt, video]
multimodal_model = GenerativeModel("gemini-1.0-pro-vision")
responses = multimodal_model.generate_content(contents, stream=True)

for response in responses:
    print(response.text, end="")

 DES stands for Data Encryption Standard. It is a symmetric-key algorithm, meaning that the same key is used to encrypt and decrypt data. DES was developed in the 1970s by IBM and was adopted as a federal standard in the United States in 1976. DES is a block cipher, meaning that it encrypts data in blocks of 64 bits at a time. The algorithm uses a 56-bit key to encrypt and decrypt data.

The DES algorithm is based on the Feistel cipher structure. The Feistel cipher structure is a type of symmetric-key block cipher that uses two keys to encrypt and decrypt data. The two keys are used to encrypt and decrypt the data in two rounds. The first round encrypts the data using the first key, and the second round decrypts the data using the second key. The Feistel cipher structure is a very secure type of cipher, and it is used in a variety of applications, including DES.

DES is a very secure algorithm, and it has been used to encrypt a wide variety of data, including financial data, medical re

In [ ]:
from google.cloud import aiplatform
import google
from vertexai import preview
import os
from dotenv import load_dotenv
from google.oauth2 import service_account
load_dotenv()
from vertexai.vision_models import (
    MultiModalEmbeddingModel,
    MultiModalEmbeddingResponse,
    Video,
    VideoSegmentConfig,
)

from vertexai.language_models import TextEmbeddingModel
#creds = os.getenv("GOOGLE_APPLICATION_CREDENTIALS")
import vertexai
from vertexai.vision_models import (
    Image,
    MultiModalEmbeddingModel,
    MultiModalEmbeddingResponse,
)
from typing import Optional
import pandas as pd
import faiss
import numpy as np
import openpyxl
import yaml
import json
import time
## SETUP
credentials = service_account.Credentials.from_service_account_file('/content/watchful-lotus-422412-h7-a2ea8eb16a41.json')
#print(creds)
aiplatform.init(

    project='watchful-lotus-422412-h7',

    location='us-central1',

    experiment='my-experiment',
    credentials=credentials,

    experiment_description='my experiment description'
)

vertexai.init(project="watchful-lotus-422412-h7", location='us-central1')



For Embeddings

In [ ]:
from vertexai.vision_models import (
    MultiModalEmbeddingModel,
    MultiModalEmbeddingResponse,
    Video,
    VideoSegmentConfig,
)

from vertexai.language_models import TextEmbeddingModel
import pandas as pd

class Embedding_Generator:

  def text_embedding(self,text) -> list:
        """Text embedding with a Large Language Model."""
        model = TextEmbeddingModel.from_pretrained("textembedding-gecko@001")
        embeddings = model.get_embeddings([text])
        for embedding in embeddings:
            vector = embedding.values
            print(f"Length of Embedding Vector: {len(vector)}")
        return vector

df = pd.read_csv("/content/be_project-dataset.csv")


data = {
    "question":"",
    "answer":"",
    "image_url":"",
    "image_caption":"",
    "video_url":"",
    "video_transcript":"",
    "source":""
}
data_map = []

for index, row in df.iterrows():
  print(index)
  data['question'] = row['Question']
  data['answer'] = row['Answer']
  data['image_url'] = row['Image']
  data['image_caption'] =row['Image_Caption']
  data['video_url']=row['Video']
  data['video_transcipt']=row['Video_Transcript']
  data['source'] = row['Source']
  data_map.append(data)
  data = {}




vectors = faiss.IndexFlatIP(1536)
text_map = []
image_map = []
idx = 0
eg = Embedding_Generator()
for row in data_map:
    retry = True
    while retry:
        try:
            text_embd = eg.text_embedding(row['question'])
            retry = False
        except Exception as e:
            print(e)
            time.sleep(10)
    text_map.append(text_embd)
    print(f"processed {idx} rows")
    idx += 1


vectors = faiss.IndexFlatIP(768)
np_list = np.asarray(text_map)
vectors.add(np_list)
faiss.write_index(vectors, f"vectors.db")
yaml.dump(data_map, open(f"datamap.yaml", "w"))

0
1
2
3
Length of Embedding Vector: 768
processed 0 rows
Length of Embedding Vector: 768
processed 1 rows
Length of Embedding Vector: 768
processed 2 rows
Length of Embedding Vector: 768
processed 3 rows


to add in database

In [ ]:
"""
text_map=[]
query_text= "what is test2?"
text_embd = eg.text_embedding(query_text)
vectors = faiss.IndexFlatIP(768)
text_map.append(text_embd)
np_list = np.asarray(text_map)
vectors.add(np_list)
all_vectors = faiss.read_index("/content/vectors.db")
all_vectors.add(vectors)
faiss.write_index(all_vectors, f"vectors.db")
"""


import yaml

yaml_path = "/content/datamap.yaml"

new_data = {
    "question":"what is test 2?",
    "answer":"idk u say . Yash is gay",
    "image_url":"",
    "image_caption":"",
    "video_url":"",
    "video_transcript":"",
    "source":""
}

with open(yaml_path, "r") as f:
  data = yaml.safe_load(f) or {}

data.append(new_data)

with open(yaml_path, "w") as f:
  yaml.dump(data, f)


In [ ]:
text_map=[]
query_text= "what is test2?"
text_embd = eg.text_embedding(query_text)
vectors = faiss.IndexFlatIP(768)
text_map.append(text_embd)
np_list = np.asarray(text_map)
vectors= faiss.read_index("/content/vectors.db")
vectors.add(np_list)
faiss.write_index(vectors, f"vectors.db")

Length of Embedding Vector: 768


TO retrieve from the database

In [ ]:
data_map=yaml.load(open("/content/datamap.yaml"), Loader=yaml.FullLoader)
vectors= faiss.read_index("/content/vectors.db")

query_question = "Who is the president of India?"
text_query_embed = eg.text_embedding(query_question)
dist, indx = vectors.search(
        np.asarray([text_query_embed], dtype=np.float32), k=5
    )
indexes = indx[0]
distances = dist[0]

context = ""
for i in range(len(indexes)):
    print(distances[i])
    print(data_map[indexes[i]]["question"])
    if distances[i] >= 0.80:
        source = data_map[indexes[i]]['source']
        answer = data_map[indexes[i]]["answer"]
        context += "\n Source: " + str(source) + str(answer) + "\n"
    else:
        context = "Nothing relevant in the Database."

# Add two newlines at the end for formatting
context += "\n\n"

Length of Embedding Vector: 768
0.5670812
test1
0.50611526
what is test 2?
0.49663955
What is the Weakness for DES 
0.49064723
What is the Pseudocode for DES cipher
0.48093787
What is shift left operation in DES?


In [ ]:
context

'Nothing relevant in the Database.\n\n'

In [ ]:
from vertexai.generative_models import (
    GenerationConfig,
    GenerativeModel,
    Image,
    Part,
)
multimodal_model = GenerativeModel("gemini-1.0-pro-vision")
question = query_question  # Replace with the actual student's question
context =context
image = "/content/aes_sub_bytes.jpg"
prompt = f"""A student is studying and has a question about the process of {question}.
They are looking at an image that depicts "{question} in the {context}".

Answer the student's question in detail, combining information from different sources and using all of it.
Explain each part in the provided image with the help of descriptions.
Make sure to include all equations and explain all examples.
"""
contents = [image, prompt]

responses = multimodal_model.generate_content(contents, stream=True)

print("-------Prompt--------")
print("\n-------Response--------")
for response in responses:
    print(response.text, end="")